### A Song of Vice and Higher: Characterizing Presidential Nominees through Game of Thrones

### Import Necessary Libraries

In [8]:
import os             # file system stuff
import json           # digest json
import pandas as pd   # Dataframes
import helper     # Custom helper functions
import gensim 
import psycopg2 as psql
from helper import attribute_comment 
from helper import person_dict

### Connect to the database

In [2]:
# Define path to secret

secret_path_aws = os.path.join(os.environ['HOME'], 'mia', '.secret', 
                           'aws_ps_flatiron.json')
# secret_path_aws = os.path.join(os.environ['HOME'], '.secret', 
#                            'aws_ps_flatiron.json')
secret_path_aws

'/Users/tjjj/mia/.secret/aws_ps_flatiron.json'

In [3]:
aws_keys = helper.get_keys(secret_path_aws)
user = aws_keys['user']
ps = aws_keys['password']
host = aws_keys['host']
db = aws_keys['db_name']

aws_ps_engine = ('postgresql://' + user + ':' + ps + '@' + host + '/' + db)

# Setup PSQL connection
conn = psql.connect(
    database=db,
    user=user,
    password=ps,
    host=host,
    port='5432'
)

In [4]:
get_database = """
    SELECT * FROM comms_sntmnt_190612;
"""
# Instantiate cursor
cur = conn.cursor()
cur.execute(get_database)
comments = pd.DataFrame(cur.fetchall())
comments.columns = [col.name for col in cur.description]
comments.drop('index', axis=1, inplace=True) #superfluous column
comments.drop(['compound', 'neu'], axis=1, inplace=True) 
#compound and neutral are correlated with other sentiment columns

In [5]:
comments.head()

,comment,domain,neg,pos
0,> after e4 i thought i'd be so angry if jaime ...,got,0.137,0.122
1,after each had spent so much time talking abou...,got,0.000,0.147
2,"after ep3 leaks came out for ep4-6, everything...",got,0.092,0.182
3,after episode 3 i had nothing by apathy for th...,got,0.000,0.127
4,"after episode 4, there really were a lot of pe...",got,0.160,0.085


## Attribute Comment to People

In [ ]:
attribute_comment(comments, person_dict)
comments.tail(10)

## Add comment length column

In [291]:
pol_df['comment_length'] = ''

In [292]:
def add_comment_length(df):
    """Add the length of the comment to our dataframe (which has a column named 'comment."""
    for index, row in pol_df.iterrows():
        pol_df.loc[index,'comment_length']= len(pol_df['comment'][index]) 

In [293]:
pol_df.head()

,comment,harris,common_words,usa,poppinkream,profanity,comment_length
0,Lawrence O'Donnell just reporting that the Tru...,0.0,10.0,0.0,0.0,0.0,
1,I saw a comment that USA Today Politics ([@usa...,0.0,147.0,17.0,1.0,0.0,
2,[deleted],0.0,0.0,0.0,0.0,0.0,
3,Holy shit anyone watching MSNBC? This asshole...,0.0,2.0,0.0,0.0,2.0,
4,Republican on CNN saying how relieved he is th...,0.0,1.0,0.0,0.0,0.0,


## Add sentiment from Werlindo's file

In [32]:
# from vaderSentiment import vaderSentiment

# analyzer = vaderSentiment.SentimentIntensityAnalyzer()

# for comment in test_comments:
#     print(comment)
#     print(analyzer.polarity_scores(comment))
# #https://github.com/cjhutto/vaderSentiment#about-the-scoring 

In [316]:
# Make comments lowercase
pol_df['comment'][0].lower()

'lawrence o\'donnell just reporting that the trump 2020 campaign committee sent an email to his supporters asking them to donate to their "official secure the border fund," and that those that donated would have their name sent to trump. immediately after the speech, a second email went out again asking for the donations. \n\nproblem: the fine print says the money goes to his reelection campaign. \n\nhe held this press conference to scoop up campaign money.'